In [ ]:
pip install gradio jupyter ipywidgets ollama

In [ ]:
import ollama
ollama.pull('mistral')

In [ ]:
from ollama import Client
client = Client(host='http://localhost:11434')
response = client.chat(model='mistral', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

In [ ]:
import ollama

stream = ollama.chat(
    model='mistral',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

In [ ]:
ollama.list()

In [ ]:
ollama.show('gemma:7b')

In [ ]:
model_to_test = 'mistral:7b'


try: 
    ollama.show(model_to_test)
    print(ollama.show(model_to_test))
except ollama.ResponseError as e:
    print(f"Pulling {model_to_test}...")
    ollama.pull(model_to_test)

In [ ]:
ollama.delete(model_to_test)

In [ ]:
import gradio as gr
from ollama import Client

def mistralChat(text):
    
    client = Client(host='http://localhost:11434')
    response = client.chat(model='mistral', messages=[
    {
        'role': 'user',
        'content': text,
    },
    ])
    return response['message']['content']

In [1]:
import gradio as gr

ollamaList = ["mistral"]
with gr.Blocks() as server:
    with gr.Row():
        with gr.Column(scale=1):
            dropdown = gr.Dropdown(choices=ollamaList)
            top_k = gr.Slider(0.0, 100.0, label="top_k", value=40, info="Reduces the probability of generating nonsense. A higher value (e.g. 100) will give more diverse answers, while a lower value (e.g. 10) will be more conservative. (Default: 40)")
            top_p = gr.Slider(0.0, 1.0, label="top_p", value=0.9, info=" Works together with top-k. A higher value (e.g., 0.95) will lead to more diverse text, while a lower value (e.g., 0.5) will generate more focused and conservative text. (Default: 0.9)")
            temp = gr.Slider(0.0, 2.0, label="temperature", value=0.8, info="The temperature of the model. Increasing the temperature will make the model answer more creatively. (Default: 0.8)")
            num_refs = gr.Slider(0, 10, label="# of References", step=1, value=1, info="The number of references to retrieve from the collection. (Default: 1)")
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(height=590)
    with gr.Row():
        with gr.Column(scale=4):
            message = gr.Textbox(placeholder="Type here")
        with gr.Column(scale=1):
            submit = gr.Button("SEND")


server.launch(server_name="0.0.0.0", server_port=8080, debug=True)

Running on local URL:  http://0.0.0.0:8080

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [ ]:
server.close()